In [2]:
!wget https://download856.mediafire.com/dja2yy6xxqzgQI7JFWbIHFXFtcIm4Vpt7-idXk_Edp_w3KECuXUnOHxb4hiOPpikwZLOE_R1alUyqv5PYkpehWxHL4u8RYRVI7jVlBGO2xg_NNJwye4rgy9OsGuxbx-w1vsVWcjNP4vw4ZmMhl33t0Jf9Rh0t4RrB96bG2G0c5Y/yte7ojclkc23syh/archive.zip

--2024-10-11 22:54:00--  https://download856.mediafire.com/dja2yy6xxqzgQI7JFWbIHFXFtcIm4Vpt7-idXk_Edp_w3KECuXUnOHxb4hiOPpikwZLOE_R1alUyqv5PYkpehWxHL4u8RYRVI7jVlBGO2xg_NNJwye4rgy9OsGuxbx-w1vsVWcjNP4vw4ZmMhl33t0Jf9Rh0t4RrB96bG2G0c5Y/yte7ojclkc23syh/archive.zip
Resolving download856.mediafire.com (download856.mediafire.com)... 205.196.121.51
Connecting to download856.mediafire.com (download856.mediafire.com)|205.196.121.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4508201844 (4.2G) [application/zip]
Saving to: ‘archive.zip’

archive.zip         100%[===================>]   4.20G  31.4MB/s    in 2m 22s  

2024-10-11 22:56:23 (30.3 MB/s) - ‘archive.zip’ saved [4508201844/4508201844]



In [3]:
!unzip archive.zip

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2863).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2864).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2865).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2866).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2867).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2868).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2869).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (287).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2870).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2871).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2872).jpg  
  inflating: ASL_Alphabet_Dataset/asl_alphabet_train/space/space (2873).jpg  
  in

In [4]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
# Ruta al directorio del dataset
data_dir = 'ASL_Alphabet_Dataset'

# Parámetros
img_height, img_width = 200, 200
batch_size = 256

# Data augmentation y preprocesamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=(0.1, 0.9),
    fill_mode='nearest',
    validation_split=0.2  # Dividir 75% para entrenamiento y 25% para validación
)

# Crear generadores
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',   # Usar el 80% de los datos para entrenamiento
    shuffle=True
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',  # Usar el 20% de los datos para validación
    shuffle=False
)

print(f"Cantidad en set de entrenamiento: {train_generator.samples}")
print(f"Cantidad en set de validación: {validation_generator.samples}")

Found 178483 images belonging to 2 classes.
Found 44619 images belonging to 2 classes.
Cantidad en set de entrenamiento: 178483
Cantidad en set de validación: 44619


In [6]:
def _conv_model():
  model = models.Sequential()
  # Primera capa convolucional
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
  model.add(layers.MaxPooling2D())

  # Segunda capa convolucional
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D())

  # Aplanamiento y capas densas
  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu'))
  model.add(layers.Dropout(0.5))  # Previene sobreajuste
  model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

  return model

In [7]:
model = _conv_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


698/698 ━━━━━━━━━━━━━━━━━━━━ 2733s 4s/step - accuracy: 0.9951 - loss: 0.0249 - val_accuracy: 0.9999 - val_loss: 0.0113
Epoch 2/20
698/698 ━━━━━━━━━━━━━━━━━━━━ 2682s 4s/step - accuracy: 0.9999 - loss: 0.0038 - val_accuracy: 0.9999 - val_loss: 0.0014
Epoch 3/20
698/698 ━━━━━━━━━━━━━━━━━━━━ 2670s 4s/step - accuracy: 0.9999 - loss: 0.0025 - val_accuracy: 0.9999 - val_loss: 0.0016
Epoch 4/20
698/698 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9999 - loss: 0.0018

KeyboardInterrupt: 